In [18]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

#LSTM Training, Test
from tensorflow.keras.layers import Embedding, Dense, LSTM
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

def data_preprocessing(data): #입력값: 외부에서 가져온 데이터셋 / 출력값: Null 제거된 데이터셋
                         #기능: 중복 제거, 한글과 공백을 제외하고 모두 제거, Null값 제거
    
    #중복 여부 검사, nunique()는 중복 제외하고 표시. list느낌
    data['document'].nunique(), data['label'].nunique()
    
    #document 컬럼에서 중복인 내용이 있으면 중복 제거
    data.drop_duplicates(subset=['document'], inplace=True)
    data = data.dropna(how='any') #Null값이 존재하는 행 제거
    
    #한글과 공백을 제외하고 모두 제거하는 정규 표현식 사용
    data['document'] = data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]", "")
    
    #다시한번 Null값이 존재하는지 확인
    data['document'] = data['document'].str.replace('^ +', "") #공백만 있거나 빈 값을 가진 행이 있으면 Null로 변경
    data['document'].replace('', np.nan, inplace=True) #Null값으로 변환
    
    #Train 데이터의 null 행 제거
    data = data.dropna(how='any')
    
    return data
    
def data_prediction(data):  #입력값: Null값이 제거된 데이터셋 / 출력값: 불용어를 제거한 후 Null값도 제거한 데이터셋
                            #기능: 불용어 제거, 단어 빈도수가 2회 이하인 단어 수를 찾아내고 공백 제거(공백 제거 목적)
    
    #불용어 사전 제작
    stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
    
    okt = Okt()
    X_data=[]

    for sentence in data['document']:
        #형태소 분석기(Okt())에서 토큰화(한글은 띄어쓰기) 실행. stem=True로 일정 수준 정규화(동사,명사화)
        X_tmp = okt.morphs(sentence, stem=True)
        X_tmp = [word for word in X_tmp if not word in stopwords] #불용어 사전에 없으면 리스트에 추가
        X_data.append(X_tmp)
       
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X_data)
    
    #등장 빈도수가 3회 미만인 단어들이 이 데이터에서 얼만큼 비중을 차지하는지 확인
    threshold = 3 #단어의 등장 빈도수 기준
    total_cnt = len(tokenizer.word_index) #단어의 수
    rare_cnt = 0 #등장 빈도수가 threshold보다 작은 단어의 개수를 카운트
    total_freq = 0 #훈련 데이터의 전체 단어 빈도수 총 합
    rare_freq = 0 #등장 빈도수가 threshold보다 작은 단어의 빈도수 총 합

    #key-value 형태로 저장
    for key, value in tokenizer.word_counts.items():
        total_freq = total_freq + value

        #단어의 등장 빈도수가 threshold보다 작으면
        if(value < threshold):
            rare_cnt = rare_cnt + 1
            rare_freq = rare_freq + value
            
    #단어 빈도수가 2회 이하인 단어들은 제외
    #0번을 고려해서 크기는 +1을 해준다.
    voca_size = total_cnt - rare_cnt + 1
    
    tokenizer = Tokenizer(voca_size)
    tokenizer.fit_on_texts(X_data)
    
    X_data = tokenizer.texts_to_sequences(X_data) #texts_to_sequences > 단어들에 순번을 지정, 0 ~ 19,415번 단어가 있음
    y_data = np.array(data['label'])
    
    #empty samples 제거
    drop_data = [index for index, sentence in enumerate(X_data) if len(sentence) < 1] #단어가 1개 미만, 즉 비어있는 데이터 제거

    #빈 샘플 제거
    X_data = np.delete(X_data, drop_data, axis=0) #X_data에서 drop_data을 사용해서 제거
    y_data = np.delete(y_data, drop_data, axis=0)
    
    #전체 훈련 데이터중 94%가 길이가 30 이하이므로 모든 샘플의 길이를 30으로 조정
    X_data = pad_sequences(X_data, maxlen = 30)
    
    return X_data, y_data, voca_size

def TrainAndTest(X_data, y_data, X_data2, y_data2): #입력값: / 출력값: /함수의 기능:
    
    model = Sequential()
    model.add(Embedding(voca_size, 100))
    model.add(LSTM(128))
    model.add(Dense(1, activation='sigmoid'))
    
    #검증 데이터 손실이 증가하면, 과적합 위험. 검증 데이터 손실이 4회 증가하면 학습을 조기 종료
    #ModelCheckpoint를 사용하여 검증 데이터의 정확도(val_acc)가 이전보다 좋아질 경우에만 모델 저장
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
    mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)
    
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
    model.fit(X_data, y_data, epochs=10, callbacks=[es, mc], batch_size=600, validation_split=0.2)
    
    loaded_model = load_model('best_model.h5')
    print("\n 테스트 정확도 : %0.4f" % (loaded_model.evaluate(X_data2, y_data2)[1]))

    
def loaded_model_def():
    return load_model('best_model.h5')
    
def sentimemt_predict(new_sentence):
    load_model = load_model_def()
    load_model = load_model('best_model.h5')
    new_sentence = okt.morphs(new_sentence, stem=True) #토큰화
    new_sentence = [word for word in new_sentence if not word in stopwords] #불용어 제거
    encoded = tokenizer.texts_to_sequences([new_sentence]) #정수 인코딩
    pad_new = pad_sequences(encoded, maxlen = 30) #패딩
    score = float(loaded_model.predict(pad_new)) #예측
    if(score > 0.5):
        print("{:.2f}% 확률로 긍정 리뷰입니다.\n".format(score * 100))
    else:
        print("{:.2f}% 확률로 부정 리뷰입니다.\n".format((1 - score) * 100))

In [2]:
train_data = pd.read_table('C:/Users/tjoeun709/Desktop/nsmc-master/ratings_train.txt')
test_data = pd.read_table('C:/Users/tjoeun709/Desktop/nsmc-master/ratings_test.txt')

In [10]:
train_data = data_preprocessing(train_data)
test_data = data_preprocessing(test_data)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:5890: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


In [11]:
train_data.head()

,id,document,label
0,9976970,아 더빙 진짜 짜증나네요 목소리,0
1,3819312,흠포스터보고 초딩영화줄오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 솔직히 재미는 없다평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화스파이더맨에서 늙어보이기만 했던 커스틴 던...,1


In [12]:
test_data.head()

,id,document,label
0,6270596,굳 ㅋ,1
2,8544678,뭐야 이 평점들은 나쁘진 않지만 점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임 돈주고 보기에는,0
4,6723715,만 아니었어도 별 다섯 개 줬을텐데 왜 로 나와서 제 심기를 불편하게 하죠,0
5,7898805,음악이 주가 된 최고의 음악영화,1


In [14]:
X_train, y_train, voca_size = data_prediction(train_data)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [16]:
X_test, y_test, v = data_prediction(test_data)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\core\_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [19]:
TrainAndTest(X_train, y_train, X_test, y_test, voca_size)

AttributeError: 'numpy.ndarray' object has no attribute 'lower'